In [2]:
import pandas as pd

In [ ]:
movies = pd.read_csv('movies.csv')

In [ ]:
r_cols = ['userId', 'movieId', 'rating', 'timestamp']
ratings = pd.read_csv('ratings.dat', delimiter='::', names=r_cols, encoding='latin-1')
ratings.rating = ratings.rating.apply(lambda x: 1 if x > 4.0 else 0)
ratings.rename(columns={'rating': 'is_click'}, inplace=True)

In [ ]:
total = movies.merge(ratings, on='movieId', how="inner")

In [33]:
movielist = list(total.movieId.value_counts()[:50])

In [34]:
total = total.loc[total.movieId.isin(movielist), :]

In [36]:
total.reset_index(drop=True, inplace=True)

In [37]:
user = list(total.sort_values(by='userId', ascending=True).userId)
user_new_index = []
user_new_index.append(0)
i = 0
for u in range(len(user)-1):
    if user[u] == user[u+1]:
        user_new_index.append(i)
    else:
        i += 1
        user_new_index.append(i)

In [38]:
userid = total.sort_values(by='userId', ascending=True)
userid['user_new_index'] = user_new_index
total = userid
total.sort_index(inplace=True)

In [39]:
item = list(total.sort_values(by='movieId', ascending=True).movieId)
movie_new_index = []
movie_new_index.append(0)
i = 0
for m in range(len(item)-1):
    if item[m] == item[m+1]:
        movie_new_index.append(i)
    else:
        i += 1
        movie_new_index.append(i)

In [40]:
movieid = total.sort_values(by='movieId', ascending=True)
movieid['movie_new_index'] = movie_new_index
total = movieid
total.sort_index(inplace=True)

In [43]:
total['user_new_index'] = total['user_new_index'].astype(int)
total['movie_new_index'] = total['movie_new_index'].astype(int)

In [45]:
total = total.sort_values(by=['user_new_index', 'timestamp'], ascending=True)

In [57]:
user_idx = list(total.user_new_index)
ts_idx = list(total.timestamp)
tmp_user = user_idx[0]
tmp_timestamp = ts_idx[0]
new_timestamp = []
idx = 0
for i in range(len(user_idx)):
    if user_idx[i] == tmp_user:
        if ts_idx[i] < tmp_timestamp + 7200: # 2시간 이내에 평가한 영화에 대해서는 같은 timestamp로 간주
            new_timestamp.append(idx)
        else:
            idx += 1
            new_timestamp.append(idx)
    else:
        idx = 0
        new_timestamp.append(idx)
    tmp_timestamp = ts_idx[i]
    tmp_user = user_idx[i]

In [59]:
total['new_timestamp'] = new_timestamp

In [61]:
total.reset_index(drop=True, inplace=True)

In [62]:
total.to_csv("data.csv", index=False)